In [1]:
import pandas as pd
import pymongo
import pprint
from collections import Counter
import numpy as np

In [2]:
from pymongo import MongoClient
client = MongoClient()

In [3]:
db = client.msbd500103

### Load Training Data

In [4]:
trData = db.training_record

In [5]:
trData = trData.find({})
trData = list(trData)
df_trData = pd.DataFrame(trData)
df_trData.head()

,_id,article_contentid,article_dwelltime,article_l1_categories,article_l2_categories,article_lan,article_publisher,article_title,article_update_time,dt,...,user_country,user_gender,user_gender_weight,user_gp_frequency,user_gp_keywords,user_id,user_maxmind_city,user_maxmind_country_iso_code,user_maxmind_state_iso_code,user_model
0,5a07224d6c0c65269105e6c7,1l5c17e72og_us,NaN,NaN,NaN,en,NaN,NaN,NaN,20170111,...,US,2.0,0.763295,"{u'SHOPPING': 1, u'BUSINESS': 1, u'ENTERTAINME...",NaN,614E304DE3B07EBCDD4E11382578EF5A,Glen Allen,US,VA,SPH-L710
1,5a07224d6c0c65269105e6c8,mQ56173a2KT_us,NaN,NaN,NaN,en,NaN,NaN,NaN,20170117,...,US,2.0,0.763295,"{u'SHOPPING': 1, u'BUSINESS': 1, u'ENTERTAINME...",NaN,614E304DE3B07EBCDD4E11382578EF5A,Glen Allen,US,VA,SPH-L710
2,5a07224d6c0c65269105e6c9,Ly7407d4cuC_us,31.0,"[{u'name': u'1000780', u'weight': 0.4546442536...","[{u'name': u'1000667', u'weight': 0.6867789988...",en,The Sun,"Dad caught selling his baby son for £9,000 to ...",1.484656e+09,20170117,...,US,2.0,0.763295,"{u'SHOPPING': 1, u'BUSINESS': 1, u'ENTERTAINME...",NaN,614E304DE3B07EBCDD4E11382578EF5A,Glen Allen,US,VA,SPH-L710
3,5a07224d6c0c65269105e6ca,AY628126a1w_us,NaN,"[{u'name': u'1000288', u'weight': 0.9748942874...","[{u'name': u'1000288', u'weight': 0.44672383556}]",en,Bustle,11 Weird Ways Your Relationship With Your Pare...,1.482379e+09,20170110,...,US,2.0,0.763295,"{u'SHOPPING': 1, u'BUSINESS': 1, u'ENTERTAINME...",NaN,614E304DE3B07EBCDD4E11382578EF5A,Glen Allen,US,VA,SPH-L710
4,5a07224d6c0c65269105e6cb,jv70536a3bt_us,NaN,"[{u'name': u'1000031', u'weight': 0.4396004700...","[{u'name': u'1000069', u'weight': 0.8568055895...",en,Los Angeles Times,Ivanka Trump goes public about exiting fashion...,1.484223e+09,20170113,...,US,2.0,0.763295,"{u'SHOPPING': 1, u'BUSINESS': 1, u'ENTERTAINME...",NaN,614E304DE3B07EBCDD4E11382578EF5A,Glen Allen,US,VA,SPH-L710


In [6]:
df_trData = df_trData[~df_trData['user_gp_frequency'].isnull()]

### Prepare User View

In [7]:
df_user = df_trData[['user_id', 'user_age', 'user_gender', 'user_brand']]#, 'user_maxmind_city']] modify sophia
df_user.head()

,user_id,user_age,user_gender,user_brand
0,614E304DE3B07EBCDD4E11382578EF5A,2.0,2.0,samsung
1,614E304DE3B07EBCDD4E11382578EF5A,2.0,2.0,samsung
2,614E304DE3B07EBCDD4E11382578EF5A,2.0,2.0,samsung
3,614E304DE3B07EBCDD4E11382578EF5A,2.0,2.0,samsung
4,614E304DE3B07EBCDD4E11382578EF5A,2.0,2.0,samsung


In [8]:
df_user['user_age'] = df_user['user_age'].fillna(0)
df_user['user_gender'] = df_user['user_gender'].fillna(0)
df_user['user_brand'] = df_user['user_brand'].str.lower().fillna('na')
#df_user['user_maxmind_city'] = df_user['user_maxmind_city'].str.lower().fillna('na') #modify sophia
df_user.head()

/Users/sophia.gao/.pyenv/versions/miniconda2-latest/lib/python2.7/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.
/Users/sophia.gao/.pyenv/versions/miniconda2-latest/lib/python2.7/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  
/Users/sophia.gao/.pyenv/versions/miniconda2-latest/lib/python2.7/site-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

,user_id,user_age,user_gender,user_brand
0,614E304DE3B07EBCDD4E11382578EF5A,2.0,2.0,samsung
1,614E304DE3B07EBCDD4E11382578EF5A,2.0,2.0,samsung
2,614E304DE3B07EBCDD4E11382578EF5A,2.0,2.0,samsung
3,614E304DE3B07EBCDD4E11382578EF5A,2.0,2.0,samsung
4,614E304DE3B07EBCDD4E11382578EF5A,2.0,2.0,samsung


In [9]:
df_user = pd.get_dummies(df_user, columns=['user_age', 'user_gender','user_brand'])#, 'user_maxmind_city']) modify sophia
df_user = df_user.fillna(0)
df_user = df_user.drop_duplicates(subset=['user_id'], keep='first')
df_user.head()

,user_id,user_age_0.0,user_age_1.0,user_age_2.0,user_age_3.0,user_age_4.0,user_gender_0.0,user_gender_1.0,user_gender_2.0,user_brand_acer,...,user_brand_tct_metropcs,user_brand_trio,user_brand_verizon,user_brand_verizon_lge,user_brand_verizon_wwe,user_brand_x325,user_brand_x401,user_brand_xiaomi,user_brand_yezz,user_brand_zte
0,614E304DE3B07EBCDD4E11382578EF5A,0,0,1,0,0,0,0,1,0,...,0,0,0,0,0,0,0,0,0,0
6,CD2F2B61135DB27BA3230F91466ED793,0,0,0,1,0,0,0,1,0,...,0,0,0,0,0,0,0,0,0,0
11,56F27AD705D8564F7900470AA793DBD2,0,0,0,0,1,0,0,1,0,...,0,0,0,0,0,0,0,0,0,0
12,8EE461BA3AA2595AC91FCDD833EC1711,0,0,1,0,0,0,0,1,0,...,0,0,0,0,0,0,0,0,0,0
15,5BA72C11B1E94BA50F1E66F4B1362CE6,0,0,1,0,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0


### Prepare App View

In [112]:
df_app = df_trData[['user_id', 'user_gp_frequency']]
df_app.head()

,user_id,user_gp_frequency
0,614E304DE3B07EBCDD4E11382578EF5A,"{u'SHOPPING': 1, u'BUSINESS': 1, u'ENTERTAINME..."
1,614E304DE3B07EBCDD4E11382578EF5A,"{u'SHOPPING': 1, u'BUSINESS': 1, u'ENTERTAINME..."
2,614E304DE3B07EBCDD4E11382578EF5A,"{u'SHOPPING': 1, u'BUSINESS': 1, u'ENTERTAINME..."
3,614E304DE3B07EBCDD4E11382578EF5A,"{u'SHOPPING': 1, u'BUSINESS': 1, u'ENTERTAINME..."
4,614E304DE3B07EBCDD4E11382578EF5A,"{u'SHOPPING': 1, u'BUSINESS': 1, u'ENTERTAINME..."


In [113]:
df_app_category = pd.DataFrame.from_records(df_app['user_gp_frequency'].tolist())
df_app_category = df_app_category.fillna(0)
df_app_category.head()

,ART_AND_DESIGN,AUTO_AND_VEHICLES,BEAUTY,BOOKS_AND_REFERENCE,BUSINESS,COMICS,COMMUNICATION,DATING,EDUCATION,ENTERTAINMENT,...,PHOTOGRAPHY,PRODUCTIVITY,SHOPPING,SOCIAL,SPORTS,TOOLS,TRANSPORTATION,TRAVEL_AND_LOCAL,VIDEO_PLAYERS,WEATHER
0,0.0,0.0,0.0,1.0,1.0,0.0,1.0,0.0,0.0,2.0,...,0.0,3.0,1.0,1.0,0.0,5.0,0.0,0.0,2.0,0.0
1,0.0,0.0,0.0,1.0,1.0,0.0,1.0,0.0,0.0,2.0,...,0.0,3.0,1.0,1.0,0.0,5.0,0.0,0.0,2.0,0.0
2,0.0,0.0,0.0,1.0,1.0,0.0,1.0,0.0,0.0,2.0,...,0.0,3.0,1.0,1.0,0.0,5.0,0.0,0.0,2.0,0.0
3,0.0,0.0,0.0,1.0,1.0,0.0,1.0,0.0,0.0,2.0,...,0.0,3.0,1.0,1.0,0.0,5.0,0.0,0.0,2.0,0.0
4,0.0,0.0,0.0,1.0,1.0,0.0,1.0,0.0,0.0,2.0,...,0.0,3.0,1.0,1.0,0.0,5.0,0.0,0.0,2.0,0.0


In [114]:
df_app = df_app.merge(df_app_category, how='left', left_index=True, right_index=True)
df_app = df_app.drop(['user_gp_frequency'], axis=1)
df_app = df_app.drop_duplicates(subset=['user_id'], keep='first')
df_app = df_app.fillna(0)
df_app.head()

,user_id,ART_AND_DESIGN,AUTO_AND_VEHICLES,BEAUTY,BOOKS_AND_REFERENCE,BUSINESS,COMICS,COMMUNICATION,DATING,EDUCATION,...,PHOTOGRAPHY,PRODUCTIVITY,SHOPPING,SOCIAL,SPORTS,TOOLS,TRANSPORTATION,TRAVEL_AND_LOCAL,VIDEO_PLAYERS,WEATHER
0,614E304DE3B07EBCDD4E11382578EF5A,0.0,0.0,0.0,1.0,1.0,0.0,1.0,0.0,0.0,...,0.0,3.0,1.0,1.0,0.0,5.0,0.0,0.0,2.0,0.0
6,CD2F2B61135DB27BA3230F91466ED793,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,...,2.0,1.0,0.0,1.0,0.0,3.0,0.0,1.0,0.0,0.0
11,56F27AD705D8564F7900470AA793DBD2,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,3.0,0.0,0.0,0.0,0.0
12,8EE461BA3AA2595AC91FCDD833EC1711,0.0,0.0,0.0,0.0,0.0,0.0,4.0,0.0,0.0,...,2.0,1.0,2.0,3.0,1.0,5.0,0.0,1.0,2.0,0.0
15,5BA72C11B1E94BA50F1E66F4B1362CE6,0.0,0.0,0.0,0.0,2.0,0.0,4.0,0.0,0.0,...,0.0,0.0,1.0,3.0,0.0,10.0,0.0,3.0,7.0,2.0


In [13]:
def sumCat(row):
    l = row.tolist()
    print l
    row['totalCat'] = sum(l[1:])
    return row

In [14]:
#df_app1 = df_app.drop(['user_id'], axis = 1).apply(lambda row: sumCat(row), axis = 1)

In [15]:
#df_app1 = df_app[['user_id']].merge(df_app1, how = 'left', left_index=True, right_index=True)

In [16]:
#df_app1.shape

In [17]:
#df_app2 = df_app1[df_app1['totalCat'] != 0]

In [18]:
#print df_app2.shape
#df_app2.head()

### Prepare Train Test Set (user-app)

In [25]:
import random
train_ratio = 0.9
num_train_user = int(round(len(df_user)*train_ratio))
train_user_id = random.sample(df_user['user_id'].tolist(), num_train_user)

In [26]:
df_train_user = df_user[df_user['user_id'].isin(train_user_id)]
df_train_user = df_train_user.drop(['user_id'], axis=1)
df_train_user = df_train_user.fillna(0)
df_train_user_arr = df_train_user.values
df_train_user_arr.shape

(7196, 54)

In [27]:
df_train_user_arr[0]

array([0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0], dtype=uint8)

In [28]:
df_test_user = df_user[~df_user['user_id'].isin(train_user_id)]
df_test_user = df_test_user.drop(['user_id'], axis=1)
df_test_user = df_test_user.fillna(0)
df_test_user_arr = df_test_user.values
df_test_user_arr.shape

(800, 54)

In [29]:
df_train_app0 = df_app[df_app['user_id'].isin(train_user_id)]
df_train_app = df_train_app0.drop(['user_id'], axis=1)
df_train_app = df_train_app.fillna(0)
df_train_app_arr = df_train_app.values
df_train_app_arr.shape

(7196, 52)

In [30]:
df_test_app0 = df_app[~df_app['user_id'].isin(train_user_id)]
df_test_app = df_test_app0.drop(['user_id'], axis=1)
df_test_app = df_test_app.fillna(0)
df_test_app_arr = df_test_app.values
df_test_app_arr.shape

(800, 52)

In [31]:
df_train_app_arr

array([[ 0.,  0.,  0., ...,  0.,  2.,  0.],
       [ 0.,  0.,  0., ...,  1.,  0.,  0.],
       [ 0.,  0.,  0., ...,  0.,  0.,  0.],
       ..., 
       [ 0.,  0.,  0., ...,  0.,  0.,  0.],
       [ 0.,  0.,  0., ...,  0.,  0.,  0.],
       [ 0.,  0.,  0., ...,  0.,  0.,  0.]])

In [32]:
y_train = [1 for i in range(len(df_train_user_arr))]
len(y_train)

7196

In [33]:
df_all_user = df_user.drop(['user_id'], axis=1)
df_all_user = df_all_user.fillna(0)
df_all_user_arr = df_all_user.values
df_all_user_arr.shape

(7996, 54)

In [34]:
df_all_app = df_app.drop(['user_id'], axis=1)
df_all_app = df_all_app.fillna(0)
df_all_app_arr = df_all_app.values
df_all_app_arr.shape

(7996, 52)

In [35]:
def sumCat(row):
    l = row.tolist()
    row['totalCat'] = sum(l)
    return row

In [36]:
df_all_app1 = df_all_app.apply(lambda row: sumCat(row), axis = 1)

In [37]:
df_all_app1.head()

,ART_AND_DESIGN,AUTO_AND_VEHICLES,BEAUTY,BOOKS_AND_REFERENCE,BUSINESS,COMICS,COMMUNICATION,DATING,EDUCATION,ENTERTAINMENT,...,PRODUCTIVITY,SHOPPING,SOCIAL,SPORTS,TOOLS,TRANSPORTATION,TRAVEL_AND_LOCAL,VIDEO_PLAYERS,WEATHER,totalCat
0,0.0,0.0,0.0,1.0,1.0,0.0,1.0,0.0,0.0,2.0,...,3.0,1.0,1.0,0.0,5.0,0.0,0.0,2.0,0.0,20.0
6,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,...,1.0,0.0,1.0,0.0,3.0,0.0,1.0,0.0,0.0,12.0
11,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,3.0,0.0,0.0,0.0,0.0,7.0
12,0.0,0.0,0.0,0.0,0.0,0.0,4.0,0.0,0.0,3.0,...,1.0,2.0,3.0,1.0,5.0,0.0,1.0,2.0,0.0,38.0
15,0.0,0.0,0.0,0.0,2.0,0.0,4.0,0.0,0.0,5.0,...,0.0,1.0,3.0,0.0,10.0,0.0,3.0,7.0,2.0,46.0


In [38]:
df_all_app1.shape

(7996, 53)

In [39]:
df_all_app2 = df_all_app1[df_all_app1['totalCat']!=0]
df_all_app2.shape

(7920, 53)

In [40]:
y_all = [1 for i in range(len(df_all_user_arr))]
len(y_all)

7996

### Build Model By Keras (User Vs App)

In [49]:
from keras.models import Sequential, Model
from keras.layers import Dense, Merge, Activation, Dropout, Reshape
import numpy as np
# fix random seed for reproducibility
np.random.seed(0)

In [50]:
df_train_user_arr.shape

(7196, 54)

In [51]:
df_train_app_arr.shape

(7196, 52)

In [52]:
df_train_app_arr[0]

array([ 0.,  0.,  0.,  1.,  1.,  0.,  1.,  0.,  0.,  2.,  0.,  0.,  0.,
        0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
        0.,  0.,  0.,  0.,  0.,  2.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
        1.,  0.,  0.,  0.,  3.,  1.,  1.,  0.,  5.,  0.,  0.,  2.,  0.])

In [53]:
y_train = [1 for i in range(len(df_train_user_arr))]
len(y_train)

7196

In [54]:
# create user model
userModel = Sequential()
'''
userModel.add(Dense(2000, input_shape=(2380,)))
userModel.add(Activation('tanh'))
userModel.add(Dropout(0.4))
userModel.add(Dense(500, input_shape=(2000,)))
userModel.add(Activation('tanh'))
userModel.add(Dropout(0.4))
userModel.add(Dense(50, input_shape=(500,)))
userModel.add(Activation('tanh'))
userModel.add(Dropout(0.4))
'''
userModel.add(Dense(30, input_shape=(54,)))
userModel.add(Activation('tanh'))
userModel.add(Dropout(0.4))
userModel.add(Dense(10, input_shape=(30,)))
userModel.add(Activation('tanh'))
userModel.add(Reshape((10,1)))

In [55]:
# create app model
appModel = Sequential()
appModel.add(Dense(30, input_shape=(52,)))
appModel.add(Activation('tanh'))
appModel.add(Dropout(0.4))
appModel.add(Dense(10, input_shape=(30,)))
appModel.add(Activation('tanh'))
appModel.add(Reshape((10,1)))

In [56]:
model = Sequential()
model.add(Merge([userModel,appModel], mode='cos', dot_axes=1, output_shape=(1,)))
model.add(Reshape((1,)))

In [57]:
# Compile model
model.compile(loss='cosine_proximity', optimizer='adam', metrics=['precision', 'recall'])

In [58]:
# Fit the model
#model.fit([df_all_user_arr, df_all_app_arr],y_all, epochs=1, batch_size=10)
model.fit([df_train_user_arr, df_train_app_arr],y_train, nb_epoch=5, batch_size=10)

Epoch 1/5
7196/7196 [==============================] - 3s - loss: -0.4208 - precision: 0.7332 - recall: 0.1604     
Epoch 2/5
7196/7196 [==============================] - 2s - loss: -0.8394 - precision: 0.9889 - recall: 0.4532     
Epoch 3/5
7196/7196 [==============================] - 2s - loss: -0.9291 - precision: 0.9986 - recall: 0.5992     
Epoch 4/5
7196/7196 [==============================] - 2s - loss: -0.9661 - precision: 1.0000 - recall: 0.7360     
Epoch 5/5
7196/7196 [==============================] - 2s - loss: -0.9744 - precision: 1.0000 - recall: 0.7689     


In [59]:
y_test = [1 for i in range(len(df_test_user_arr))]
len(y_test)

800

In [60]:
# evaluate the model
scores = model.evaluate([df_test_user_arr,df_test_app_arr], y_test)
print("\n%s: %.2f%%" % (model.metrics_names[1], scores[1]*100))

 32/800 [>.............................] - ETA: 0s
precision: 100.00%


In [61]:
model.predict([df_test_user_arr,df_test_app_arr], batch_size=10, verbose=0)

array([[ 0.64977628],
       [ 0.9395085 ],
       [ 0.8798483 ],
       [ 0.61445767],
       [ 0.81029534],
       [ 0.91483992],
       [ 0.56276786],
       [ 0.90835464],
       [ 0.70825279],
       [ 0.94720381],
       [ 0.86558014],
       [ 0.8875407 ],
       [ 0.85610861],
       [ 0.93470484],
       [ 0.65806353],
       [ 0.74345881],
       [ 0.90268505],
       [ 0.91613179],
       [ 0.92861289],
       [ 0.89014363],
       [ 0.85881108],
       [ 0.8239398 ],
       [ 0.73758054],
       [ 0.8387168 ],
       [ 0.76833099],
       [ 0.57877171],
       [ 0.84888971],
       [ 0.91004676],
       [ 0.78162688],
       [ 0.91246909],
       [ 0.9413175 ],
       [ 0.94300723],
       [ 0.8803823 ],
       [ 0.88904452],
       [ 0.91238171],
       [ 0.92849082],
       [ 0.88881207],
       [ 0.93412465],
       [ 0.90181637],
       [ 0.91478753],
       [ 0.90470207],
       [ 0.00754108],
       [ 0.8648051 ],
       [ 0.79988194],
       [ 0.92203641],
       [ 0

### Prepare Article View

In [77]:
df_article = df_trData[['user_id', 'article_contentid', 'article_l1_categories', 'article_l2_categories', 'article_publisher', 'article_title']]
df_article = df_article.sort_values(by=['article_title'], ascending=False)
df_article.head()

,user_id,article_contentid,article_l1_categories,article_l2_categories,article_publisher,article_title
73696,7C62CF4B33EF7303881DB654E68FF85E,v77c577e1qo_us,"[{u'name': u'1000780', u'weight': 0.8786688469...","[{u'name': u'1000667', u'weight': 0.4648142188...",Daily Record,﻿Vulnerable teen 'livestreams suicide' on Face...
125132,A4F138114721E738AD8C2768C11450A9,v77c577e1qo_us,"[{u'name': u'1000780', u'weight': 0.8786688469...","[{u'name': u'1000667', u'weight': 0.4648142188...",Daily Record,﻿Vulnerable teen 'livestreams suicide' on Face...
45894,986759E2FD77D818BEDC17BFF6376900,v77c577e1qo_us,"[{u'name': u'1000780', u'weight': 0.8786688469...","[{u'name': u'1000667', u'weight': 0.4648142188...",Daily Record,﻿Vulnerable teen 'livestreams suicide' on Face...
136028,17A177FDE5C3C9F500D784FF8D985AD7,0i65cb6eaNb_us,"[{u'name': u'1000780', u'weight': 0.0405336841...","[{u'name': u'1000780', u'weight': 0.0405336841...",芸能・バラエティ,【E-girls】ちび丸子ちゃん―踊りを学びましょう～
136080,17A177FDE5C3C9F500D784FF8D985AD7,0i65cb6eaNb_us,"[{u'name': u'1000780', u'weight': 0.0405336841...","[{u'name': u'1000780', u'weight': 0.0405336841...",芸能・バラエティ,【E-girls】ちび丸子ちゃん―踊りを学びましょう～


In [78]:
df_article = df_article.drop_duplicates(subset=['article_contentid'], keep='first')
df_article = df_article.reset_index(drop = True)
df_article.head()

,user_id,article_contentid,article_l1_categories,article_l2_categories,article_publisher,article_title
0,7C62CF4B33EF7303881DB654E68FF85E,v77c577e1qo_us,"[{u'name': u'1000780', u'weight': 0.8786688469...","[{u'name': u'1000667', u'weight': 0.4648142188...",Daily Record,﻿Vulnerable teen 'livestreams suicide' on Face...
1,17A177FDE5C3C9F500D784FF8D985AD7,0i65cb6eaNb_us,"[{u'name': u'1000780', u'weight': 0.0405336841...","[{u'name': u'1000780', u'weight': 0.0405336841...",芸能・バラエティ,【E-girls】ちび丸子ちゃん―踊りを学びましょう～
2,DB7995E79922E8A96E1585DA8C6EFB51,ag639e0eeDK_us,"[{u'name': u'8000001', u'weight': 0.999}]","[{u'name': u'8000001', u'weight': 0.999}]",Human Fail,♠ Funny Videos ♠ Best FAILS ♠ December 2016 Co...
3,84E7779E484630375F5F0F6175045B4B,7m799a2c4mX_us,"[{u'name': u'1000931', u'weight': 0.1268057078...","[{u'name': u'1000931', u'weight': 0.1268057078...",NighTcore FC,▶[Future Bass] ★ MRVLZ - On The Inside
4,DE83F55126D386CF852A76F25FE0FE04,hQ7cfd994Tv_us,"[{u'name': u'1000780', u'weight': 0.4859073658...","[{u'name': u'1000780', u'weight': 0.4859073658...",Limerick Leader,€1m in funding sought for new air ambulance se...


In [79]:
def getDetail(cat, field):
    if (str(cat) != 'nan'):
        if (len(cat) > 0):
            return cat[0][field]
    else:
        return '0'

In [80]:
df_article['l1_cat'] = df_article.apply(lambda row: getDetail(row['article_l1_categories'],'name'), axis=1)
df_article['l2_cat'] = df_article.apply(lambda row: getDetail(row['article_l2_categories'],'name'), axis=1)
df_article['l1_cat_w'] = df_article.apply(lambda row: getDetail(row['article_l1_categories'],'weight'), axis=1)
df_article['l2_cat_w'] = df_article.apply(lambda row: getDetail(row['article_l2_categories'],'weight'), axis=1)
df_article = df_article.drop(['article_l1_categories', 'article_l2_categories'], axis=1)
df_article.head()

,user_id,article_contentid,article_publisher,article_title,l1_cat,l2_cat,l1_cat_w,l2_cat_w
0,7C62CF4B33EF7303881DB654E68FF85E,v77c577e1qo_us,Daily Record,﻿Vulnerable teen 'livestreams suicide' on Face...,1000780,1000667,0.878669,0.464814
1,17A177FDE5C3C9F500D784FF8D985AD7,0i65cb6eaNb_us,芸能・バラエティ,【E-girls】ちび丸子ちゃん―踊りを学びましょう～,1000780,1000780,0.0405337,0.0405337
2,DB7995E79922E8A96E1585DA8C6EFB51,ag639e0eeDK_us,Human Fail,♠ Funny Videos ♠ Best FAILS ♠ December 2016 Co...,8000001,8000001,0.999,0.999
3,84E7779E484630375F5F0F6175045B4B,7m799a2c4mX_us,NighTcore FC,▶[Future Bass] ★ MRVLZ - On The Inside,1000931,1000931,0.126806,0.126806
4,DE83F55126D386CF852A76F25FE0FE04,hQ7cfd994Tv_us,Limerick Leader,€1m in funding sought for new air ambulance se...,1000780,1000780,0.485907,0.485907


### Clean title

In [81]:
df_article = df_article[~df_article['article_title'].isnull()]
df_article.head()

,user_id,article_contentid,article_publisher,article_title,l1_cat,l2_cat,l1_cat_w,l2_cat_w
0,7C62CF4B33EF7303881DB654E68FF85E,v77c577e1qo_us,Daily Record,﻿Vulnerable teen 'livestreams suicide' on Face...,1000780,1000667,0.878669,0.464814
1,17A177FDE5C3C9F500D784FF8D985AD7,0i65cb6eaNb_us,芸能・バラエティ,【E-girls】ちび丸子ちゃん―踊りを学びましょう～,1000780,1000780,0.0405337,0.0405337
2,DB7995E79922E8A96E1585DA8C6EFB51,ag639e0eeDK_us,Human Fail,♠ Funny Videos ♠ Best FAILS ♠ December 2016 Co...,8000001,8000001,0.999,0.999
3,84E7779E484630375F5F0F6175045B4B,7m799a2c4mX_us,NighTcore FC,▶[Future Bass] ★ MRVLZ - On The Inside,1000931,1000931,0.126806,0.126806
4,DE83F55126D386CF852A76F25FE0FE04,hQ7cfd994Tv_us,Limerick Leader,€1m in funding sought for new air ambulance se...,1000780,1000780,0.485907,0.485907


In [82]:
from gensim.models import Word2Vec
from nltk.corpus import stopwords
from nltk import word_tokenize
from nltk.stem.lancaster import LancasterStemmer
import re

In [83]:
def hashingTitle(title):
    STEMMER = LancasterStemmer()
    if (type(title) != float) & (type(title) != int):
        title = title.lower()
        title = word_tokenize(title)
        title = [STEMMER.stem(word.strip('!"#$%&\()*+,-./:;<=>?@[\]^_`{|}~').lower()) for word in title]
        titleList = []
        for w in title:
            if len(re.findall("[a-zA-Z]+", w)) > 0:
                f_w = re.findall("[a-zA-Z]+", w)[0].lower()
                f_w = '#'+f_w+'#'
                f_w_tri = [f_w[i:i+3] for i in range(len(f_w)-1) if (len(f_w[i:i+3]) == 3)]
                titleList.extend(f_w_tri)
        return dict(Counter(titleList))
    else:
        return {}

In [84]:
df_article['clean_article_title'] = df_article.apply(lambda x: hashingTitle(x['article_title']), axis=1)
df_article.head()

,user_id,article_contentid,article_publisher,article_title,l1_cat,l2_cat,l1_cat_w,l2_cat_w,clean_article_title
0,7C62CF4B33EF7303881DB654E68FF85E,v77c577e1qo_us,Daily Record,﻿Vulnerable teen 'livestreams suicide' on Face...,1000780,1000667,0.878669,0.464814,"{u'om#': 2, u'ive': 1, u'uic': 1, u'le#': 1, u..."
1,17A177FDE5C3C9F500D784FF8D985AD7,0i65cb6eaNb_us,芸能・バラエティ,【E-girls】ちび丸子ちゃん―踊りを学びましょう～,1000780,1000780,0.0405337,0.0405337,{u'#e#': 1}
2,DB7995E79922E8A96E1585DA8C6EFB51,ag639e0eeDK_us,Human Fail,♠ Funny Videos ♠ Best FAILS ♠ December 2016 Co...,8000001,8000001,0.999,0.999,"{u'anf': 1, u'pil': 1, u'vid': 1, u'#fa': 1, u..."
3,84E7779E484630375F5F0F6175045B4B,7m799a2c4mX_us,NighTcore FC,▶[Future Bass] ★ MRVLZ - On The Inside,1000931,1000931,0.126806,0.126806,"{u'ass': 1, u'nsi': 1, u'#ba': 1, u'rvl': 1, u..."
4,DE83F55126D386CF852A76F25FE0FE04,hQ7cfd994Tv_us,Limerick Leader,€1m in funding sought for new air ambulance se...,1000780,1000780,0.485907,0.485907,"{u'rv#': 1, u'#fo': 1, u'ugh': 1, u'ht#': 1, u..."


In [85]:
df_article_title = pd.DataFrame.from_records(df_article['clean_article_title'].tolist())
df_article_title = df_article_title.fillna(0)
df_article_title.head()

,#a#,#aa,#ab,#ac,#ad,#ae,#af,#ag,#ah,#ai,...,zyt,zz#,zza,zze,zzf,zzi,zzl,zzn,zzo,zzy
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [86]:
df_article = df_article.merge(df_article_title, how='left', left_index=True, right_index=True)
df_article.head()

,user_id,article_contentid,article_publisher,article_title,l1_cat,l2_cat,l1_cat_w,l2_cat_w,clean_article_title,#a#,...,zyt,zz#,zza,zze,zzf,zzi,zzl,zzn,zzo,zzy
0,7C62CF4B33EF7303881DB654E68FF85E,v77c577e1qo_us,Daily Record,﻿Vulnerable teen 'livestreams suicide' on Face...,1000780,1000667,0.878669,0.464814,"{u'om#': 2, u'ive': 1, u'uic': 1, u'le#': 1, u...",0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,17A177FDE5C3C9F500D784FF8D985AD7,0i65cb6eaNb_us,芸能・バラエティ,【E-girls】ちび丸子ちゃん―踊りを学びましょう～,1000780,1000780,0.0405337,0.0405337,{u'#e#': 1},0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,DB7995E79922E8A96E1585DA8C6EFB51,ag639e0eeDK_us,Human Fail,♠ Funny Videos ♠ Best FAILS ♠ December 2016 Co...,8000001,8000001,0.999,0.999,"{u'anf': 1, u'pil': 1, u'vid': 1, u'#fa': 1, u...",0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,84E7779E484630375F5F0F6175045B4B,7m799a2c4mX_us,NighTcore FC,▶[Future Bass] ★ MRVLZ - On The Inside,1000931,1000931,0.126806,0.126806,"{u'ass': 1, u'nsi': 1, u'#ba': 1, u'rvl': 1, u...",0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,DE83F55126D386CF852A76F25FE0FE04,hQ7cfd994Tv_us,Limerick Leader,€1m in funding sought for new air ambulance se...,1000780,1000780,0.485907,0.485907,"{u'rv#': 1, u'#fo': 1, u'ugh': 1, u'ht#': 1, u...",0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [87]:
df_article = df_article.drop(['article_title'], axis=1)
#df_article = df_article.drop(['article_title', 'clean_article_title'], axis=1)
df_article.head()

,user_id,article_contentid,article_publisher,l1_cat,l2_cat,l1_cat_w,l2_cat_w,clean_article_title,#a#,#aa,...,zyt,zz#,zza,zze,zzf,zzi,zzl,zzn,zzo,zzy
0,7C62CF4B33EF7303881DB654E68FF85E,v77c577e1qo_us,Daily Record,1000780,1000667,0.878669,0.464814,"{u'om#': 2, u'ive': 1, u'uic': 1, u'le#': 1, u...",0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,17A177FDE5C3C9F500D784FF8D985AD7,0i65cb6eaNb_us,芸能・バラエティ,1000780,1000780,0.0405337,0.0405337,{u'#e#': 1},0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,DB7995E79922E8A96E1585DA8C6EFB51,ag639e0eeDK_us,Human Fail,8000001,8000001,0.999,0.999,"{u'anf': 1, u'pil': 1, u'vid': 1, u'#fa': 1, u...",0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,84E7779E484630375F5F0F6175045B4B,7m799a2c4mX_us,NighTcore FC,1000931,1000931,0.126806,0.126806,"{u'ass': 1, u'nsi': 1, u'#ba': 1, u'rvl': 1, u...",0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,DE83F55126D386CF852A76F25FE0FE04,hQ7cfd994Tv_us,Limerick Leader,1000780,1000780,0.485907,0.485907,"{u'rv#': 1, u'#fo': 1, u'ugh': 1, u'ht#': 1, u...",0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [88]:
df_article = pd.get_dummies(df_article, columns=['article_publisher'])
df_article = df_article.fillna(0)
df_article.head()

,user_id,article_contentid,l1_cat,l2_cat,l1_cat_w,l2_cat_w,clean_article_title,#a#,#aa,#ab,...,article_publisher_yahoo,article_publisher_yahoo.com,article_publisher_ynetnews.com,article_publisher_youforanimal,article_publisher_youralaskalink.com,article_publisher_zacktv1,article_publisher_zimnews.net,article_publisher_کەناڵی ئێن ئاڕ تی - سەرەتا,article_publisher_– IMEMC News,article_publisher_芸能・バラエティ
0,7C62CF4B33EF7303881DB654E68FF85E,v77c577e1qo_us,1000780,1000667,0.878669,0.464814,"{u'om#': 2, u'ive': 1, u'uic': 1, u'le#': 1, u...",0.0,0.0,0.0,...,0,0,0,0,0,0,0,0,0,0
1,17A177FDE5C3C9F500D784FF8D985AD7,0i65cb6eaNb_us,1000780,1000780,0.040534,0.040534,{u'#e#': 1},0.0,0.0,0.0,...,0,0,0,0,0,0,0,0,0,1
2,DB7995E79922E8A96E1585DA8C6EFB51,ag639e0eeDK_us,8000001,8000001,0.999000,0.999000,"{u'anf': 1, u'pil': 1, u'vid': 1, u'#fa': 1, u...",0.0,0.0,0.0,...,0,0,0,0,0,0,0,0,0,0
3,84E7779E484630375F5F0F6175045B4B,7m799a2c4mX_us,1000931,1000931,0.126806,0.126806,"{u'ass': 1, u'nsi': 1, u'#ba': 1, u'rvl': 1, u...",0.0,0.0,0.0,...,0,0,0,0,0,0,0,0,0,0
4,DE83F55126D386CF852A76F25FE0FE04,hQ7cfd994Tv_us,1000780,1000780,0.485907,0.485907,"{u'rv#': 1, u'#fo': 1, u'ugh': 1, u'ht#': 1, u...",0.0,0.0,0.0,...,0,0,0,0,0,0,0,0,0,0


In [89]:
df_cat = df_article[['l1_cat', 'l2_cat', 'l1_cat_w', 'l2_cat_w']]
df_cat = pd.get_dummies(df_cat, columns=['l1_cat', 'l2_cat'])
df_cat = df_cat.fillna(0)
df_cat = df_cat.drop(['l1_cat_0', 'l2_cat_0'], axis=1)
df_cat.head()

,l1_cat_w,l2_cat_w,l1_cat_0010000000,l1_cat_1000001,l1_cat_1000031,l1_cat_1000044,l1_cat_1000111,l1_cat_1000123,l1_cat_1000267,l1_cat_1000288,...,l2_cat_1001040,l2_cat_1001049,l2_cat_1001060,l2_cat_1001091,l2_cat_1001103,l2_cat_1001110,l2_cat_1001117,l2_cat_2000000,l2_cat_6000007,l2_cat_8000001
0,0.878669,0.464814,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0.040534,0.040534,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0.999000,0.999000,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
3,0.126806,0.126806,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0.485907,0.485907,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [90]:
df_cat = df_cat.merge(df_article[['l1_cat', 'l2_cat']], how='left', left_index=True, right_index=True)
df_cat.head()

,l1_cat_w,l2_cat_w,l1_cat_0010000000,l1_cat_1000001,l1_cat_1000031,l1_cat_1000044,l1_cat_1000111,l1_cat_1000123,l1_cat_1000267,l1_cat_1000288,...,l2_cat_1001060,l2_cat_1001091,l2_cat_1001103,l2_cat_1001110,l2_cat_1001117,l2_cat_2000000,l2_cat_6000007,l2_cat_8000001,l1_cat,l2_cat
0,0.878669,0.464814,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1000780,1000667
1,0.040534,0.040534,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1000780,1000780
2,0.999000,0.999000,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,1,8000001,8000001
3,0.126806,0.126806,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1000931,1000931
4,0.485907,0.485907,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1000780,1000780


In [91]:
def updateCatWeight(row):
    cat = row['l1_cat']
    row['l1_cat_'+str(cat)] = row['l1_cat_w']
    return row

In [92]:
df_cat = df_cat.apply(lambda row: updateCatWeight(row), axis=1)
df_cat.head()

,l1_cat,l1_cat_0,l1_cat_0010000000,l1_cat_1000001,l1_cat_1000031,l1_cat_1000044,l1_cat_1000111,l1_cat_1000123,l1_cat_1000267,l1_cat_1000288,...,l2_cat_1001049,l2_cat_1001060,l2_cat_1001091,l2_cat_1001103,l2_cat_1001110,l2_cat_1001117,l2_cat_2000000,l2_cat_6000007,l2_cat_8000001,l2_cat_w
0,1000780,NaN,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0,0,0,0,0,0,0,0,0,0.464814
1,1000780,NaN,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0,0,0,0,0,0,0,0,0,0.040534
2,8000001,NaN,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0,0,0,0,0,0,0,0,1,0.999000
3,1000931,NaN,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0,0,0,0,0,0,0,0,0,0.126806
4,1000780,NaN,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0,0,0,0,0,0,0,0,0,0.485907


In [93]:
df_cat = df_cat.drop(['l1_cat', 'l2_cat', 'l1_cat_0'], axis=1)
df_cat.head()

,l1_cat_0010000000,l1_cat_1000001,l1_cat_1000031,l1_cat_1000044,l1_cat_1000111,l1_cat_1000123,l1_cat_1000267,l1_cat_1000288,l1_cat_1000298,l1_cat_1000374,...,l2_cat_1001049,l2_cat_1001060,l2_cat_1001091,l2_cat_1001103,l2_cat_1001110,l2_cat_1001117,l2_cat_2000000,l2_cat_6000007,l2_cat_8000001,l2_cat_w
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0,0,0,0,0,0,0,0,0,0.464814
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0,0,0,0,0,0,0,0,0,0.040534
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0,0,0,0,0,0,0,0,1,0.999000
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0,0,0,0,0,0,0,0,0,0.126806
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0,0,0,0,0,0,0,0,0,0.485907


In [94]:
df_article = df_article.merge(df_cat, how='left', left_index=True, right_index=True)
df_article.head()

,user_id,article_contentid,l1_cat,l2_cat,l1_cat_w_x,l2_cat_w_x,clean_article_title,#a#,#aa,#ab,...,l2_cat_1001049,l2_cat_1001060,l2_cat_1001091,l2_cat_1001103,l2_cat_1001110,l2_cat_1001117,l2_cat_2000000,l2_cat_6000007,l2_cat_8000001,l2_cat_w_y
0,7C62CF4B33EF7303881DB654E68FF85E,v77c577e1qo_us,1000780,1000667,0.878669,0.464814,"{u'om#': 2, u'ive': 1, u'uic': 1, u'le#': 1, u...",0.0,0.0,0.0,...,0,0,0,0,0,0,0,0,0,0.464814
1,17A177FDE5C3C9F500D784FF8D985AD7,0i65cb6eaNb_us,1000780,1000780,0.040534,0.040534,{u'#e#': 1},0.0,0.0,0.0,...,0,0,0,0,0,0,0,0,0,0.040534
2,DB7995E79922E8A96E1585DA8C6EFB51,ag639e0eeDK_us,8000001,8000001,0.999000,0.999000,"{u'anf': 1, u'pil': 1, u'vid': 1, u'#fa': 1, u...",0.0,0.0,0.0,...,0,0,0,0,0,0,0,0,1,0.999000
3,84E7779E484630375F5F0F6175045B4B,7m799a2c4mX_us,1000931,1000931,0.126806,0.126806,"{u'ass': 1, u'nsi': 1, u'#ba': 1, u'rvl': 1, u...",0.0,0.0,0.0,...,0,0,0,0,0,0,0,0,0,0.126806
4,DE83F55126D386CF852A76F25FE0FE04,hQ7cfd994Tv_us,1000780,1000780,0.485907,0.485907,"{u'rv#': 1, u'#fo': 1, u'ugh': 1, u'ht#': 1, u...",0.0,0.0,0.0,...,0,0,0,0,0,0,0,0,0,0.485907


In [95]:
#df_article = df_article.drop(['article_title', 'article_contentid', 'l1_cat', 'l2_cat', 'l1_cat_w', 'l2_cat_w'], axis = 1)
df_article.head()

,user_id,article_contentid,l1_cat,l2_cat,l1_cat_w_x,l2_cat_w_x,clean_article_title,#a#,#aa,#ab,...,l2_cat_1001049,l2_cat_1001060,l2_cat_1001091,l2_cat_1001103,l2_cat_1001110,l2_cat_1001117,l2_cat_2000000,l2_cat_6000007,l2_cat_8000001,l2_cat_w_y
0,7C62CF4B33EF7303881DB654E68FF85E,v77c577e1qo_us,1000780,1000667,0.878669,0.464814,"{u'om#': 2, u'ive': 1, u'uic': 1, u'le#': 1, u...",0.0,0.0,0.0,...,0,0,0,0,0,0,0,0,0,0.464814
1,17A177FDE5C3C9F500D784FF8D985AD7,0i65cb6eaNb_us,1000780,1000780,0.040534,0.040534,{u'#e#': 1},0.0,0.0,0.0,...,0,0,0,0,0,0,0,0,0,0.040534
2,DB7995E79922E8A96E1585DA8C6EFB51,ag639e0eeDK_us,8000001,8000001,0.999000,0.999000,"{u'anf': 1, u'pil': 1, u'vid': 1, u'#fa': 1, u...",0.0,0.0,0.0,...,0,0,0,0,0,0,0,0,1,0.999000
3,84E7779E484630375F5F0F6175045B4B,7m799a2c4mX_us,1000931,1000931,0.126806,0.126806,"{u'ass': 1, u'nsi': 1, u'#ba': 1, u'rvl': 1, u...",0.0,0.0,0.0,...,0,0,0,0,0,0,0,0,0,0.126806
4,DE83F55126D386CF852A76F25FE0FE04,hQ7cfd994Tv_us,1000780,1000780,0.485907,0.485907,"{u'rv#': 1, u'#fo': 1, u'ugh': 1, u'ht#': 1, u...",0.0,0.0,0.0,...,0,0,0,0,0,0,0,0,0,0.485907


In [96]:
df_user.head()

,user_id,user_age_0.0,user_age_1.0,user_age_2.0,user_age_3.0,user_age_4.0,user_gender_0.0,user_gender_1.0,user_gender_2.0,user_brand_acer,...,user_brand_tct_metropcs,user_brand_trio,user_brand_verizon,user_brand_verizon_lge,user_brand_verizon_wwe,user_brand_x325,user_brand_x401,user_brand_xiaomi,user_brand_yezz,user_brand_zte
0,614E304DE3B07EBCDD4E11382578EF5A,0,0,1,0,0,0,0,1,0,...,0,0,0,0,0,0,0,0,0,0
6,CD2F2B61135DB27BA3230F91466ED793,0,0,0,1,0,0,0,1,0,...,0,0,0,0,0,0,0,0,0,0
11,56F27AD705D8564F7900470AA793DBD2,0,0,0,0,1,0,0,1,0,...,0,0,0,0,0,0,0,0,0,0
12,8EE461BA3AA2595AC91FCDD833EC1711,0,0,1,0,0,0,0,1,0,...,0,0,0,0,0,0,0,0,0,0
15,5BA72C11B1E94BA50F1E66F4B1362CE6,0,0,1,0,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0


In [97]:
df_user_article = df_user.merge(df_article, how='left', on='user_id')
df_user_article.head()

,user_id,user_age_0.0,user_age_1.0,user_age_2.0,user_age_3.0,user_age_4.0,user_gender_0.0,user_gender_1.0,user_gender_2.0,user_brand_acer,...,l2_cat_1001049,l2_cat_1001060,l2_cat_1001091,l2_cat_1001103,l2_cat_1001110,l2_cat_1001117,l2_cat_2000000,l2_cat_6000007,l2_cat_8000001,l2_cat_w_y
0,614E304DE3B07EBCDD4E11382578EF5A,0,0,1,0,0,0,0,1,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.856806
1,614E304DE3B07EBCDD4E11382578EF5A,0,0,1,0,0,0,0,1,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.686779
2,CD2F2B61135DB27BA3230F91466ED793,0,0,0,1,0,0,0,1,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,56F27AD705D8564F7900470AA793DBD2,0,0,0,0,1,0,0,1,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,8EE461BA3AA2595AC91FCDD833EC1711,0,0,1,0,0,0,0,1,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [98]:
df_user_article = df_user_article.fillna(0)
df_user_article = df_user_article[(df_user_article['l1_cat'] != 0) & (df_user_article['l2_cat'] != 0)]
df_user_article.head()

,user_id,user_age_0.0,user_age_1.0,user_age_2.0,user_age_3.0,user_age_4.0,user_gender_0.0,user_gender_1.0,user_gender_2.0,user_brand_acer,...,l2_cat_1001049,l2_cat_1001060,l2_cat_1001091,l2_cat_1001103,l2_cat_1001110,l2_cat_1001117,l2_cat_2000000,l2_cat_6000007,l2_cat_8000001,l2_cat_w_y
0,614E304DE3B07EBCDD4E11382578EF5A,0,0,1,0,0,0,0,1,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.856806
1,614E304DE3B07EBCDD4E11382578EF5A,0,0,1,0,0,0,0,1,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.686779
5,5BA72C11B1E94BA50F1E66F4B1362CE6,0,0,1,0,0,0,1,0,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.362159
6,B3B51127D5FB5B39E4273BE65CACA28C,0,0,1,0,0,0,0,1,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.583826
8,E1295F6622EA8DA36DE678772D9ED152,0,0,1,0,0,0,0,1,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.739064


In [100]:
df_ua_user = df_user_article[df_user.columns]
#df_ua_user = df_ua_user.drop(['user_id'], axis=1)
df_ua_user = df_ua_user.fillna(0)
df_ua_user.head()

,user_id,user_age_0.0,user_age_1.0,user_age_2.0,user_age_3.0,user_age_4.0,user_gender_0.0,user_gender_1.0,user_gender_2.0,user_brand_acer,...,user_brand_tct_metropcs,user_brand_trio,user_brand_verizon,user_brand_verizon_lge,user_brand_verizon_wwe,user_brand_x325,user_brand_x401,user_brand_xiaomi,user_brand_yezz,user_brand_zte
0,614E304DE3B07EBCDD4E11382578EF5A,0,0,1,0,0,0,0,1,0,...,0,0,0,0,0,0,0,0,0,0
1,614E304DE3B07EBCDD4E11382578EF5A,0,0,1,0,0,0,0,1,0,...,0,0,0,0,0,0,0,0,0,0
5,5BA72C11B1E94BA50F1E66F4B1362CE6,0,0,1,0,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0
6,B3B51127D5FB5B39E4273BE65CACA28C,0,0,1,0,0,0,0,1,0,...,0,0,0,0,0,0,0,0,0,0
8,E1295F6622EA8DA36DE678772D9ED152,0,0,1,0,0,0,0,1,0,...,0,0,0,0,0,0,0,0,0,1


In [101]:
df_ua_article = df_user_article[df_article.columns]
#df_ua_article = df_ua_article.drop(['user_id', 'article_contentid'], axis=1)
#df_ua_article = df_ua_article.drop(['article_contentid'], axis=1)# modify by sophia
df_ua_article = df_ua_article.fillna(0)
df_ua_article.head()

,user_id,article_contentid,l1_cat,l2_cat,l1_cat_w_x,l2_cat_w_x,clean_article_title,#a#,#aa,#ab,...,l2_cat_1001049,l2_cat_1001060,l2_cat_1001091,l2_cat_1001103,l2_cat_1001110,l2_cat_1001117,l2_cat_2000000,l2_cat_6000007,l2_cat_8000001,l2_cat_w_y
0,614E304DE3B07EBCDD4E11382578EF5A,jv70536a3bt_us,1000031,1000069,0.439600,0.856806,"{u'and': 1, u'nk#': 1, u'iva': 1, u'#go': 1, u...",0.0,0.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.856806
1,614E304DE3B07EBCDD4E11382578EF5A,Ly7407d4cuC_us,1000780,1000667,0.454644,0.686779,"{u'and': 1, u'wif': 1, u'ugh': 1, u'#fo': 1, u...",2.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.686779
5,5BA72C11B1E94BA50F1E66F4B1362CE6,fP7b7115agK_us,1000288,1000288,0.362159,0.362159,"{u'#mo': 1, u'whe': 1, u'her': 1, u'om#': 1, u...",0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.362159
6,B3B51127D5FB5B39E4273BE65CACA28C,sr715100efS_us,1000780,1000780,0.817986,0.583826,"{u'vic': 1, u'oss': 1, u'vil': 1, u'#re': 1, u...",0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.583826
8,E1295F6622EA8DA36DE678772D9ED152,ZD782d17023_us,1000031,1000069,0.844815,0.739064,"{u'sew': 1, u'all': 1, u'eal': 1, u'sal': 1, u...",0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.739064


In [115]:
df_upa_article = df_ua_article.merge(df_app, how = 'left', on = 'user_id')
print df_upa_article.shape
df_upa_article.head()

(49129, 14847)


,user_id,article_contentid,clean_article_title,#a#,#aa,#ab,#ac,#ad,#ae,#af,...,PHOTOGRAPHY,PRODUCTIVITY,SHOPPING,SOCIAL,SPORTS,TOOLS,TRANSPORTATION,TRAVEL_AND_LOCAL,VIDEO_PLAYERS,WEATHER
0,614E304DE3B07EBCDD4E11382578EF5A,jv70536a3bt_us,"{u'and': 1, u'nk#': 1, u'iva': 1, u'#go': 1, u...",0.0,0.0,1.0,0.0,0.0,0.0,0.0,...,0.0,3.0,1.0,1.0,0.0,5.0,0.0,0.0,2.0,0.0
1,614E304DE3B07EBCDD4E11382578EF5A,Ly7407d4cuC_us,"{u'and': 1, u'wif': 1, u'ugh': 1, u'#fo': 1, u...",2.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,3.0,1.0,1.0,0.0,5.0,0.0,0.0,2.0,0.0
2,5BA72C11B1E94BA50F1E66F4B1362CE6,fP7b7115agK_us,"{u'#mo': 1, u'whe': 1, u'her': 1, u'om#': 1, u...",0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,1.0,3.0,0.0,10.0,0.0,3.0,7.0,2.0
3,B3B51127D5FB5B39E4273BE65CACA28C,sr715100efS_us,"{u'vic': 1, u'oss': 1, u'vil': 1, u'#re': 1, u...",0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,1.0,1.0,0.0,2.0,0.0,3.0,0.0,1.0,1.0,0.0
4,E1295F6622EA8DA36DE678772D9ED152,ZD782d17023_us,"{u'sew': 1, u'all': 1, u'eal': 1, u'sal': 1, u...",0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,2.0,4.0,0.0,2.0,0.0,16.0,0.0,1.0,1.0,0.0


### Prepare Train Test Set (user-article)

In [116]:
import random
train_ratio = 0.9
num_train_user = int(round(len(df_ua_user)*train_ratio))
train_user_id = random.sample(df_ua_user['user_id'].tolist(), num_train_user)

In [117]:
df_train_user = df_ua_user[df_ua_user['user_id'].isin(train_user_id)]
df_train_user = df_train_user.drop(['user_id'], axis=1)
df_train_user = df_train_user.fillna(0)
df_train_user_arr = df_train_user.values
df_train_user_arr.shape

(48946, 54)

In [118]:
df_test_user0 = df_ua_user[~df_ua_user['user_id'].isin(train_user_id)]
df_test_user = df_test_user0.drop(['user_id'], axis=1)
df_test_user = df_test_user.fillna(0)
df_test_user_arr = df_test_user.values
df_test_user_arr.shape

(183, 54)

In [121]:
#df_ua_article = df_ua_article.drop(['l1_cat', 'l2_cat', 'l1_cat_w_x', 'l2_cat_w_x', 'l1_cat_w_y', 'l2_cat_w_y'], axis=1)

In [106]:
cat_col = [i for i in df_ua_article.columns if ("l1_cat" in i) or ("l2_cat" in i)]
len(cat_col)

181

In [107]:
df_train_article = df_ua_article[df_ua_article['user_id'].isin(train_user_id)]

df_train_app0 = df_train_article[['user_id']].merge(df_app, how = 'left', on = 'user_id')
df_train_app = df_train_app0.drop(['user_id'], axis=1)
df_train_app = df_train_app.fillna(0)
df_train_app_arr = df_train_app.values
print df_train_app_arr.shape

(48940, 52)


In [ ]:
df_train_article = df_train_article.drop(['user_id', 'article_contentid'], axis=1)# modify by sophia
df_train_article = df_train_article.fillna(0)
df_train_article = df_train_article[cat_col]
df_train_article_arr = df_train_article.values
print df_train_article_arr.shape

In [ ]:
df_test_article0 = df_ua_article[~df_ua_article['user_id'].isin(train_user_id)]
df_test_article = df_test_article0.drop(['user_id', 'article_contentid'], axis=1)# modify by sophia
df_test_article = df_test_article.fillna(0)
df_test_article = df_test_article[cat_col]
df_test_article_arr = df_test_article.values
df_test_article_arr.shape

In [ ]:
df_test_app0 = df_test_article0[['user_id']].merge(df_app, how = 'left', on = 'user_id')
df_test_app = df_test_app0.drop(['user_id'], axis=1)
df_test_app = df_test_app.fillna(0)
df_test_app_arr = df_test_app.values
df_test_app_arr.shape

In [ ]:
y_train = [1 for i in range(len(df_train_user_arr))]
len(y_train)

### Build Model By Keras (User Vs Article)

In [63]:
from keras.models import Sequential, Model
from keras.layers import Dense, Merge, Activation, Dropout, Reshape
import numpy as np
# fix random seed for reproducibility
np.random.seed(0)

In [64]:
df_train_user_arr.shape

(48941, 54)

In [65]:
df_train_article_arr.shape

(48941, 181)

In [66]:
y_train = [1 for i in range(len(df_train_user_arr))]
len(y_train)

48941

In [67]:
df_train_user_arr

array([[0, 0, 1, ..., 0, 0, 0],
       [0, 0, 1, ..., 0, 0, 0],
       [0, 0, 1, ..., 0, 0, 0],
       ..., 
       [0, 0, 1, ..., 0, 0, 0],
       [0, 0, 1, ..., 0, 0, 0],
       [1, 0, 0, ..., 0, 0, 0]], dtype=uint8)

In [68]:
# create user model
userModel = Sequential()
'''
userModel.add(Dense(2000, input_shape=(2380,)))
userModel.add(Activation('tanh'))
userModel.add(Dropout(0.4))
userModel.add(Dense(500, input_shape=(2000,)))
userModel.add(Activation('tanh'))
userModel.add(Dropout(0.4))
userModel.add(Dense(50, input_shape=(500,)))
userModel.add(Activation('tanh'))
userModel.add(Dropout(0.4))
'''
userModel.add(Dense(30, input_shape=(54,)))
userModel.add(Activation('tanh'))
userModel.add(Dropout(0.4))
userModel.add(Dense(10, input_shape=(30,)))
userModel.add(Activation('tanh'))
userModel.add(Reshape((10,1)))

In [69]:
# create app model
articleModel = Sequential()
articleModel.add(Dense(50, input_shape=(181,)))
articleModel.add(Activation('tanh'))
articleModel.add(Dense(30, input_shape=(50,)))
articleModel.add(Activation('tanh'))
articleModel.add(Dense(10, input_shape=(30,)))
articleModel.add(Activation('tanh'))
articleModel.add(Reshape((10,1)))

In [70]:
model = Sequential()
model.add(Merge([userModel,articleModel], mode='cos', dot_axes=1, output_shape=(1,)))
model.add(Reshape((1,)))

In [71]:
# Compile model
model.compile(loss='cosine_proximity', optimizer='adam', metrics=['accuracy'])#metrics = ['precision', 'recall']

In [72]:
df_train_user_arr[0]

array([0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0], dtype=uint8)

In [73]:
df_train_article_arr[0:3]
print df_train_article_arr.shape, df_train_user_arr.shape

(48941, 181) (48941, 54)


In [74]:
# Fit the model
#model.fit([df_all_user_arr, df_all_app_arr],y_all, epochs=1, batch_size=10)
model.fit([df_train_user_arr, df_train_article_arr],y_train, nb_epoch=1, batch_size=10)

Epoch 1/1
48941/48941 [==============================] - 18s - loss: -0.9523 - acc: 0.8741    


In [75]:
y_test = [1 for i in range(len(df_test_user_arr))]
len(y_test)

188

In [76]:
# evaluate the model
scores = model.evaluate([df_test_user_arr,df_test_article_arr], y_test)
print("\n%s: %.2f%%" % (model.metrics_names[1], scores[1]*100))

 32/188 [====>.........................] - ETA: 0s
acc: 100.00%


In [77]:
model.predict([df_test_user_arr,df_test_article_arr], batch_size=10, verbose=0)

array([[ 0.96274406],
       [ 0.93508452],
       [ 0.94851971],
       [ 0.9621526 ],
       [ 0.95747983],
       [ 0.91191059],
       [ 0.96159327],
       [ 0.96163213],
       [ 0.94584805],
       [ 0.93350714],
       [ 0.9690181 ],
       [ 0.9855231 ],
       [ 0.96567696],
       [ 0.96972096],
       [ 0.96159166],
       [ 0.98738021],
       [ 0.87844533],
       [ 0.95870101],
       [ 0.97821683],
       [ 0.90995467],
       [ 0.97812021],
       [ 0.93054634],
       [ 0.84981471],
       [ 0.93712521],
       [ 0.94627172],
       [ 0.90764236],
       [ 0.94002849],
       [ 0.97481072],
       [ 0.98742485],
       [ 0.9660573 ],
       [ 0.97850364],
       [ 0.97413141],
       [ 0.79131132],
       [ 0.93772757],
       [ 0.95813239],
       [ 0.920892  ],
       [ 0.95706391],
       [ 0.86519557],
       [ 0.93673581],
       [ 0.9773286 ],
       [ 0.91808593],
       [ 0.92730641],
       [ 0.95784038],
       [ 0.94834822],
       [ 0.94986987],
       [ 0

### Calculate precision/recall on a single user

In [86]:
# Test user list
df_test_user0['user_id'].iloc[1]

u'D79CC4620095EAF4E21BA529129E8107'

In [87]:
# Predict on the first user
# Get User ID
uid = df_test_user0['user_id'].iloc[1]
# Get prediction score
l = len(df_test_article_arr)
u = df_test_user_arr[2]
temp = np.array([u] * l)
up = pd.DataFrame(model.predict([temp,df_test_article_arr], batch_size=10, verbose=0))
df_test_article1=df_test_article0.reset_index(drop = True)\
                                .merge(up, how = 'left', left_index=True, right_index=True)\
                                .rename(columns={0:'u0_score'})
df_test_article1.head()

,user_id,article_contentid,#a#,#aa,#ab,#ac,#ad,#ae,#af,#ag,...,l2_cat_1001049,l2_cat_1001060,l2_cat_1001091,l2_cat_1001103,l2_cat_1001110,l2_cat_1001117,l2_cat_2000000,l2_cat_6000007,l2_cat_8000001,u0_score
0,026F279414571C9A841F8E8E86EC594D,gJ74e851cxO_us,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.953069
1,D79CC4620095EAF4E21BA529129E8107,9S6f5b7602s_us,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.959182
2,40929E249813D17DE0BE52F61CDDCD62,G47044b49k2_us,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.948520
3,D2CCEC6C0F772D9D14C239DF1F3531B7,rp7a4dd7d1F_us,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.958920
4,DC7E5807E0CF9B47EFE3ECA461866A73,kE7dcda4bKn_us,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.964808


In [88]:
# add y_pred column
df_test_article1 = df_test_article1[['article_contentid', 'u0_score']]
df_test_article1 = df_test_article1.sort_values(by='u0_score', ascending=False)
df_test_article1['y_pred'] = 0
df_test_article1[:5]['y_pred']=1

/Users/sophia.gao/.pyenv/versions/miniconda2-latest/lib/python2.7/site-packages/ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """


In [89]:
df_test_article1.head()

,article_contentid,u0_score,y_pred
31,aH6b9c8e1hC_us,0.988923,1
8,nC7d7c6f9BA_us,0.983207,1
163,Mb703635d7R_us,0.982275,1
169,aL737544dUD_us,0.982265,1
116,vE79161e3QI_us,0.979194,1


In [90]:
# add y_true column
df_match = df_test_article0.ix[df_test_article0['user_id'] == uid]#,['user_id','readList']]
df_match = df_match[['article_contentid']]
df_match['y_true'] = 1
df_match.head()

,article_contentid,y_true
488,9S6f5b7602s_us,1


In [91]:
df_test_article1 = df_test_article1.merge(df_match, how = 'left', on = 'article_contentid')
df_test_article1 = df_test_article1.fillna(0)
df_test_article1.head()

,article_contentid,u0_score,y_pred,y_true
0,aH6b9c8e1hC_us,0.988923,1,0.0
1,nC7d7c6f9BA_us,0.983207,1,0.0
2,Mb703635d7R_us,0.982275,1,0.0
3,aL737544dUD_us,0.982265,1,0.0
4,vE79161e3QI_us,0.979194,1,0.0


In [92]:
'''
y_true = df_test_article1['y_true'].tolist()
y_pred = df_test_article1['y_pred'].tolist()

# use sklearn function : this will include prediction of 0
from sklearn.metrics import classification_report
target_names = ['Yread', 'Nread']
print(classification_report(y_true, y_pred, target_names=target_names))
'''

"\ny_true = df_test_article1['y_true'].tolist()\ny_pred = df_test_article1['y_pred'].tolist()\n\n# use sklearn function : this will include prediction of 0\nfrom sklearn.metrics import classification_report\ntarget_names = ['Yread', 'Nread']\nprint(classification_report(y_true, y_pred, target_names=target_names))\n"

In [93]:
tp = df_test_article1.ix[(df_test_article1['y_pred'] == 1) & (df_test_article1['y_true'] == 1)].shape[0]
fp = df_test_article1.ix[(df_test_article1['y_pred'] == 1) & (df_test_article1['y_true'] == 0)].shape[0]
fn = df_test_article1.ix[(df_test_article1['y_pred'] == 0) & (df_test_article1['y_true'] == 1)].shape[0]
precision = tp/(float(tp+fp))
recall = tp/(float(tp+fn))
precision, recall

/Users/sophia.gao/.pyenv/versions/miniconda2-latest/lib/python2.7/site-packages/ipykernel_launcher.py:1: DeprecationWarning: 
.ix is deprecated. Please use
.loc for label based indexing or
.iloc for positional indexing

See the documentation here:
http://pandas.pydata.org/pandas-docs/stable/indexing.html#ix-indexer-is-deprecated
  """Entry point for launching an IPython kernel.


(0.0, 0.0)

### Calculate precision and recall

In [219]:
def calPrecitionRecall(i, model, df_test_user0, df_test_article0, len_test_article, df_test_article_arr):
    #users = df_test_user0['user_id'].tolist()
    precision = list()
    recall = list()
    for idx, user in enumerate(df_test_user_arr):
        uid = df_test_user0['user_id'].tolist()[idx]
        user = df_test_user_arr[idx]
        temp = np.array([user] * len_test_article)
        up = pd.DataFrame(model.predict([temp,df_test_article_arr], batch_size=10, verbose=0))
        df_test_article1=df_test_article0.reset_index(drop = True)\
                                .merge(up, how = 'left', left_index=True, right_index=True)\
                                .rename(columns={0:'u0_score'})[['article_contentid', 'u0_score']]
        df_test_article1 = df_test_article1.sort_values(by='u0_score', ascending=False)
        df_test_article1['y_pred'] = 0
        df_test_article1[:5]['y_pred']=1
        df_match = df_test_article0.ix[df_test_article0['user_id'] == uid]#,['user_id','readList']]
        df_match = df_match[['article_contentid']]
        df_match['y_true'] = 1
        df_test_article1 = df_test_article1.merge(df_match, how = 'left', on = 'article_contentid')\
                                           .fillna(0)
        tp = df_test_article1.ix[(df_test_article1['y_pred'] == 1) & (df_test_article1['y_true'] == 1)].shape[0]
        fp = df_test_article1.ix[(df_test_article1['y_pred'] == 1) & (df_test_article1['y_true'] == 0)].shape[0]
        fn = df_test_article1.ix[(df_test_article1['y_pred'] == 0) & (df_test_article1['y_true'] == 1)].shape[0]
        p = tp/(float(tp+fp))
        r = tp/(float(tp+fn))
        print idx, uid, p,r
        if p != 0:
            precision.append(p)
        if r != 0:
            recall.append(r)
    return precision, recall
l = len(df_test_article_arr)
calPrecitionRecall(20, model, df_test_user0, df_test_article0, l, df_test_article_arr)

/Users/sophia.gao/.pyenv/versions/miniconda2-latest/lib/python2.7/site-packages/ipykernel_launcher.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  del sys.path[0]
/Users/sophia.gao/.pyenv/versions/miniconda2-latest/lib/python2.7/site-packages/ipykernel_launcher.py:18: DeprecationWarning: 
.ix is deprecated. Please use
.loc for label based indexing or
.iloc for positional indexing

See the documentation here:
http://pandas.pydata.org/pandas-docs/stable/indexing.html#ix-indexer-is-deprecated


0 EBFC388E0E9DD470826DC38191DE7C9B 0.0 0.0
1 7457F64B04A8CA8ABE129CCA69829883 0.2 1.0
2 8125BE615FDC19E33A83D5F07B1C85A7 0.0 0.0
3 D6D22AAC72A65BDCAA11FF3604153B91 0.0 0.0
4 C6F60A318E6B6D1E5B254A941A0D6CBB 0.0 0.0
5 BF52EAC20839519075CEFC6D5926223C 0.0 0.0
6 8CBC349F15C06823CB9CAB21C2C4DB6C 0.0 0.0
7 A5559642A3E7CAE2489322828D85259F 0.0 0.0
8 373CCE3D0D1C589EDF55DEA89CAF13B6 0.0 0.0
9 A824C8F6E565621BE2C93CF2DB46AC76 0.0 0.0
10 12E971948898EDA684E823DE7605CC8F 0.0 0.0
11 7BBECF4C6987F306BFD75F9988479688 0.0 0.0
12 7BBECF4C6987F306BFD75F9988479688 0.0 0.0
13 F92A745B77D82A9860CF6FD06F87387A 0.0 0.0
14 AA4552E26FB6B1454B1B165902DD1870 0.0 0.0
15 774CAC5D49C02AB006D58E53286795B7 0.0 0.0
16 DE9F5688ED1FD4C27D65A836AB1C917E 0.0 0.0
17 1DF8AAA3E2E5FB00D28A9FF6D8A39428 0.0 0.0
18 0EE541BCAB458C86E7B4339C3A0CA63F 0.0 0.0
19 775734EC8ABE0ADD991C9BDBC16C4798 0.0 0.0
20 744AC6BAE96053959C00D7AA6BB175ED 0.0 0.0
21 B6D353CDCAFBFC93B7E03DE67970ABCD 0.0 0.0
22 3D20C7201721FA241E819E9A6CA82C74 0.0 0.

([], [])

In [94]:
def calPrecitionRecall0(model, df_test_user0, df_test_article0, len_test_article, df_test_article_arr):
    #users = df_test_user0['user_id'].tolist()
    precision = list()
    recall = list()
    for idx, user in enumerate(df_test_user_arr):
        uid = df_test_user0['user_id'].tolist()[idx]
        user = df_test_user_arr[idx]
        temp = np.array([user] * len_test_article)
        up = pd.DataFrame(model.predict([temp,df_test_article_arr], batch_size=10, verbose=0))
        df_test_article1=df_test_article0.reset_index(drop = True)\
                                .merge(up, how = 'left', left_index=True, right_index=True)\
                                .rename(columns={0:'u0_score'})[['article_contentid', 'u0_score']]
        df_test_article1 = df_test_article1.sort_values(by='u0_score', ascending=False)
        df_test_article1['y_pred'] = 0
        df_test_article1[:5]['y_pred']=1
        df_match = df_test_article0.ix[df_test_article0['user_id'] == uid]#,['user_id','readList']]
        df_match = df_match[['article_contentid']]
        df_match['y_true'] = 1
        df_test_article1 = df_test_article1.merge(df_match, how = 'left', on = 'article_contentid')\
                                           .fillna(0)
        tp = df_test_article1.ix[(df_test_article1['y_pred'] == 1) & (df_test_article1['y_true'] == 1)].shape[0]
        fp = df_test_article1.ix[(df_test_article1['y_pred'] == 1) & (df_test_article1['y_true'] == 0)].shape[0]
        fn = df_test_article1.ix[(df_test_article1['y_pred'] == 0) & (df_test_article1['y_true'] == 1)].shape[0]
        p = tp/(float(tp+fp))
        r = tp/(float(tp+fn))
        print idx, uid, p,r
        if p != 0:
            precision.append(p)
        if r != 0:
            recall.append(r)
    return precision, recall
l = len(df_test_article_arr)
calPrecitionRecall0( model, df_test_user0, df_test_article0, l, df_test_article_arr)

/Users/sophia.gao/.pyenv/versions/miniconda2-latest/lib/python2.7/site-packages/ipykernel_launcher.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  del sys.path[0]
/Users/sophia.gao/.pyenv/versions/miniconda2-latest/lib/python2.7/site-packages/ipykernel_launcher.py:18: DeprecationWarning: 
.ix is deprecated. Please use
.loc for label based indexing or
.iloc for positional indexing

See the documentation here:
http://pandas.pydata.org/pandas-docs/stable/indexing.html#ix-indexer-is-deprecated


0 026F279414571C9A841F8E8E86EC594D 0.0 0.0
1 D79CC4620095EAF4E21BA529129E8107 0.0 0.0
2 40929E249813D17DE0BE52F61CDDCD62 0.0 0.0
3 D2CCEC6C0F772D9D14C239DF1F3531B7 0.0 0.0
4 DC7E5807E0CF9B47EFE3ECA461866A73 0.0 0.0
5 2F35FC28EE00DDA023AA6C51431F4255 0.0 0.0
6 D7B0FB107E5A4FB08628AE90F1D218A7 0.0 0.0
7 D7B0FB107E5A4FB08628AE90F1D218A7 0.0 0.0
8 21B99C00321192C844381B757C61FCB5 0.0 0.0
9 159836F8BD4EC2793DF59B9DECF53E19 0.0 0.0
10 5D6BFBB1E09DEF03B3787AC88711A071 0.0 0.0
11 5D6BFBB1E09DEF03B3787AC88711A071 0.0 0.0
12 387AE98BC9826FD0058F6D3B383F2107 0.0 0.0
13 70AB056DB89A69FB4A82B80EAB0AA5D1 0.0 0.0
14 1BD20ED43E9F13069577CCAC5DF512A0 0.0 0.0
15 1282A2EFF2DFF65534085CDFAC82F94B 0.0 0.0
16 EAF2FA1C3FC905E1345B953DCC6C628A 0.0 0.0
17 7CDC1DD1CFFE8A02D33BA72DE64EE587 0.0 0.0
18 1FB61068629C9034270F6D9909E77BEF 0.0 0.0
19 744AC6BAE96053959C00D7AA6BB175ED 0.0 0.0
20 32933BB4A132FEF360E1A9CAB6BAE4B6 0.0 0.0
21 756BF8545BFC79B78DA6CDC7DD4FE53D 0.0 0.0
22 C3DB0D42505350215C0899299A01CC73 0.0 0.

185 3E84D636D9A3E39F9E993DFBFCEFCF6B 0.0 0.0
186 5FD0CA1E363C3D9C5FD29DE8CF88779D 0.0 0.0
187 2AA2847282A745B356261E8D43E783E4 0.0 0.0


([0.2, 0.2], [1.0, 1.0])

###### For the model of user feature (without city) - user article (use category features only)
precition: 0.2128%
recall: 1.0638%

38BC0EA4CB4C08BC7645756CE07AD8B3

DE5E9F5367C825437FC32749C637E02A

###### For the model of user feature (with city) - user article (use category features only)
precition: 0.5952%
recall: 2.976%

7457F64B04A8CA8ABE129CCA69829883

4F626685464E106FC92DD6EBB8AD1BA6

88428EBEC07BE695A711358B414C82F8

586BBFDF09881FD6B42CFC3BBD82037C

B64563330542D612206C172E19C9EFFD

### Prepare Train Test Set (app-article)

In [88]:
import random
train_ratio = 0.9
num_train_user = int(round(len(df_ua_user)*train_ratio))
train_user_id = random.sample(df_ua_user['user_id'].tolist(), num_train_user)

In [120]:
df_ua_user.head()

,user_id,user_age_0.0,user_age_1.0,user_age_2.0,user_age_3.0,user_age_4.0,user_gender_0.0,user_gender_1.0,user_gender_2.0,user_brand_acer,...,user_brand_tct_metropcs,user_brand_trio,user_brand_verizon,user_brand_verizon_lge,user_brand_verizon_wwe,user_brand_x325,user_brand_x401,user_brand_xiaomi,user_brand_yezz,user_brand_zte
0,614E304DE3B07EBCDD4E11382578EF5A,0,0,1,0,0,0,0,1,0,...,0,0,0,0,0,0,0,0,0,0
1,614E304DE3B07EBCDD4E11382578EF5A,0,0,1,0,0,0,0,1,0,...,0,0,0,0,0,0,0,0,0,0
2,614E304DE3B07EBCDD4E11382578EF5A,0,0,1,0,0,0,0,1,0,...,0,0,0,0,0,0,0,0,0,0
3,614E304DE3B07EBCDD4E11382578EF5A,0,0,1,0,0,0,0,1,0,...,0,0,0,0,0,0,0,0,0,0
7,5BA72C11B1E94BA50F1E66F4B1362CE6,0,0,1,0,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0


In [89]:
df_train_user = df_ua_user[df_ua_user['user_id'].isin(train_user_id)]
df_train_user = df_train_user.drop(['user_id'], axis=1)
df_train_user = df_train_user.fillna(0)
df_train_user_arr = df_train_user.values
df_train_user_arr.shape

(53713, 54)

In [90]:
df_test_user0 = df_ua_user[~df_ua_user['user_id'].isin(train_user_id)]
df_test_user = df_test_user0.drop(['user_id'], axis=1)
df_test_user = df_test_user.fillna(0)
df_test_user_arr = df_test_user.values
df_test_user_arr.shape

(203, 54)

In [93]:
#df_ua_article = df_ua_article.drop(['l1_cat', 'l2_cat', 'l1_cat_w_x', 'l2_cat_w_x', 'l1_cat_w_y', 'l2_cat_w_y'], axis=1)

In [94]:
cat_col = [i for i in df_ua_article.columns if ("l1_cat" in i) or ("l2_cat" in i)]
len(cat_col)

181

In [95]:
df_train_article = df_ua_article[df_ua_article['user_id'].isin(train_user_id)]

df_train_app0 = df_train_article[['user_id']].merge(df_app, how = 'left', on = 'user_id')
df_train_app = df_train_app0.drop(['user_id'], axis=1)
df_train_app = df_train_app.fillna(0)
df_train_app_arr = df_train_app.values
print df_train_app_arr.shape

(53713, 52)


In [96]:
df_train_article = df_train_article.drop(['user_id', 'article_contentid'], axis=1)# modify by sophia
df_train_article = df_train_article.fillna(0)
df_train_article = df_train_article[cat_col]
df_train_article_arr = df_train_article.values
print df_train_article_arr.shape

(53713, 181)


In [97]:
df_test_article0 = df_ua_article[~df_ua_article['user_id'].isin(train_user_id)]
df_test_article = df_test_article0.drop(['user_id', 'article_contentid'], axis=1)# modify by sophia
df_test_article = df_test_article.fillna(0)
df_test_article = df_test_article[cat_col]
df_test_article_arr = df_test_article.values
df_test_article_arr.shape

(203, 181)

In [98]:
df_test_app0 = df_test_article0[['user_id']].merge(df_app, how = 'left', on = 'user_id')
df_test_app = df_test_app0.drop(['user_id'], axis=1)
df_test_app = df_test_app.fillna(0)
df_test_app_arr = df_test_app.values
df_test_app_arr.shape

(203, 52)

In [99]:
y_train = [1 for i in range(len(df_train_user_arr))]
len(y_train)

53713

### Build Model By Keras (App Vs Article)

In [100]:
from keras.models import Sequential, Model
from keras.layers import Dense, Merge, Activation, Dropout, Reshape
import numpy as np
# fix random seed for reproducibility
np.random.seed(0)

In [101]:
df_train_app_arr.shape

(53713, 52)

In [102]:
df_train_article_arr.shape

(53713, 181)

In [103]:
y_train = [1 for i in range(len(df_train_user_arr))]
len(y_train)

53713

In [104]:
df_train_app_arr

array([[ 0.,  0.,  0., ...,  0.,  2.,  0.],
       [ 0.,  0.,  0., ...,  0.,  2.,  0.],
       [ 0.,  0.,  0., ...,  0.,  2.,  0.],
       ..., 
       [ 0.,  0.,  0., ...,  0.,  0.,  0.],
       [ 0.,  0.,  0., ...,  0.,  0.,  0.],
       [ 0.,  0.,  0., ...,  0.,  0.,  0.]])

In [105]:
# create user model
userModel = Sequential()
userModel.add(Dense(30, input_shape=(52,)))
userModel.add(Activation('tanh'))
userModel.add(Dropout(0.4))
userModel.add(Dense(10, input_shape=(30,)))
userModel.add(Activation('tanh'))
userModel.add(Reshape((10,1)))

In [106]:
# create app model
'''
articleModel = Sequential()
articleModel.add(Dense(10, input_shape=(27,)))
articleModel.add(Activation('tanh'))
articleModel.add(Reshape((10,1)))
'''
# create app model
articleModel = Sequential()
articleModel.add(Dense(50, input_shape=(181,)))
articleModel.add(Activation('tanh'))
articleModel.add(Dense(30, input_shape=(50,)))
articleModel.add(Activation('tanh'))
articleModel.add(Dense(10, input_shape=(30,)))
articleModel.add(Activation('tanh'))
articleModel.add(Reshape((10,1)))

In [107]:
model = Sequential()
model.add(Merge([userModel,articleModel], mode='cos', dot_axes=1, output_shape=(1,)))
model.add(Reshape((1,)))

In [108]:
# Compile model
model.compile(loss='cosine_proximity', optimizer='adam', metrics=['accuracy'])

In [109]:
df_train_app_arr[0]

array([ 0.,  0.,  0.,  1.,  1.,  0.,  1.,  0.,  0.,  2.,  0.,  0.,  0.,
        0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
        0.,  0.,  0.,  0.,  0.,  2.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
        1.,  0.,  0.,  0.,  3.,  1.,  1.,  0.,  5.,  0.,  0.,  2.,  0.])

In [114]:
df_train_article_arr[0:3]
print df_train_article_arr.shape, df_train_app_arr.shape

(53713, 181) (53713, 52)


In [111]:
# Fit the model
#model.fit([df_all_user_arr, df_all_app_arr],y_all, epochs=1, batch_size=10)
model.fit([df_train_app_arr, df_train_article_arr],y_train, nb_epoch=1, batch_size=10)

Epoch 1/1
53713/53713 [==============================] - 24s - loss: nan - acc: 0.0000e+00    


In [116]:
y_test = [1 for i in range(len(df_test_app_arr))]
len(y_test)

203

In [117]:
# evaluate the model
scores = model.evaluate([df_test_app_arr,df_test_article_arr], y_test)
print("\n%s: %.2f%%" % (model.metrics_names[1], scores[1]*100))

 32/203 [===>..........................] - ETA: 0s
acc: 0.00%


In [119]:
model.predict([df_test_app_arr,df_test_article_arr], batch_size=10, verbose=0)

array([[ nan],
       [ nan],
       [ nan],
       [ nan],
       [ nan],
       [ nan],
       [ nan],
       [ nan],
       [ nan],
       [ nan],
       [ nan],
       [ nan],
       [ nan],
       [ nan],
       [ nan],
       [ nan],
       [ nan],
       [ nan],
       [ nan],
       [ nan],
       [ nan],
       [ nan],
       [ nan],
       [ nan],
       [ nan],
       [ nan],
       [ nan],
       [ nan],
       [ nan],
       [ nan],
       [ nan],
       [ nan],
       [ nan],
       [ nan],
       [ nan],
       [ nan],
       [ nan],
       [ nan],
       [ nan],
       [ nan],
       [ nan],
       [ nan],
       [ nan],
       [ nan],
       [ nan],
       [ nan],
       [ nan],
       [ nan],
       [ nan],
       [ nan],
       [ nan],
       [ nan],
       [ nan],
       [ nan],
       [ nan],
       [ nan],
       [ nan],
       [ nan],
       [ nan],
       [ nan],
       [ nan],
       [ nan],
       [ nan],
       [ nan],
       [ nan],
       [ nan],
       [ n

In [116]:
def calPrecitionRecall_aa(i, model, df_test_user0, df_test_article0, len_test_article, df_test_article_arr):
    #users = df_test_user0['user_id'].tolist()
    precision = list()
    recall = list()
    for idx, user in enumerate(df_test_user_arr):
        uid = df_test_user0['user_id'].tolist()[idx]
        user = df_test_user_arr[idx]
        temp = np.array([user] * len_test_article)
        up = pd.DataFrame(model.predict([temp,df_test_article_arr], batch_size=10, verbose=0))
        df_test_article1=df_test_article0.reset_index(drop = True)\
                                .merge(up, how = 'left', left_index=True, right_index=True)\
                                .rename(columns={0:'u0_score'})[['article_contentid', 'u0_score']]
        df_test_article1 = df_test_article1.sort_values(by='u0_score', ascending=False)
        df_test_article1['y_pred'] = 0
        df_test_article1[:5]['y_pred']=1
        df_match = df_test_article0.ix[df_test_article0['user_id'] == uid]#,['user_id','readList']]
        df_match = df_match[['article_contentid']]
        df_match['y_true'] = 1
        df_test_article1 = df_test_article1.merge(df_match, how = 'left', on = 'article_contentid')\
                                           .fillna(0)
        tp = df_test_article1.ix[(df_test_article1['y_pred'] == 1) & (df_test_article1['y_true'] == 1)].shape[0]
        fp = df_test_article1.ix[(df_test_article1['y_pred'] == 1) & (df_test_article1['y_true'] == 0)].shape[0]
        fn = df_test_article1.ix[(df_test_article1['y_pred'] == 0) & (df_test_article1['y_true'] == 1)].shape[0]
        p = tp/(float(tp+fp))
        r = tp/(float(tp+fn))
        print idx, uid, p,r
        if p != 0:
            precision.append(p)
        if r != 0:
            recall.append(r)
    return precision, recall
l = len(df_test_article_arr)
calPrecitionRecall_aa(20, model, df_test_user0, df_test_article0, l, df_test_article_arr)

[[ 0.94997013]
 [ 0.94930285]
 [ 0.94922113]
 [ 0.94910455]
 [ 0.94871253]
 [ 0.94776618]
 [ 0.94703376]
 [ 0.94684452]
 [ 0.94663721]
 [ 0.94643593]
 [ 0.94243085]
 [ 0.94218558]
 [ 0.9416191 ]
 [ 0.94160342]
 [ 0.9409703 ]
 [ 0.94057965]
 [ 0.94030851]
 [ 0.93967861]
 [ 0.93945324]
 [ 0.93919587]
 [ 0.93885738]
 [ 0.93793994]
 [ 0.93790317]
 [ 0.93764073]
 [ 0.93741995]
 [ 0.93672395]
 [ 0.93666303]
 [ 0.93654752]
 [ 0.92486322]
 [ 0.92483789]
 [ 0.92251998]
 [ 0.92040735]
 [ 0.91958213]
 [ 0.91862601]
 [ 0.91824353]
 [ 0.91824353]
 [ 0.91824353]
 [ 0.91824353]
 [ 0.91824353]
 [ 0.91824353]
 [ 0.91824353]
 [ 0.91824353]
 [ 0.91824353]
 [ 0.91824353]
 [ 0.91824353]
 [ 0.91824353]
 [ 0.91824353]
 [ 0.91824353]
 [ 0.91824353]
 [ 0.91824353]
 [ 0.91824353]
 [ 0.91824353]
 [ 0.91609097]
 [ 0.90936238]
 [ 0.90519607]
 [ 0.90392429]
 [ 0.90256876]
 [ 0.9016093 ]
 [ 0.9004274 ]
 [ 0.89944834]
 [ 0.89678478]
 [ 0.89636105]
 [ 0.89534634]
 [ 0.89458996]
 [ 0.89388639]
 [ 0.89304936]
 [ 0.89303

### Build Model By Keras (User Vs App Vs Article)

In [ ]:
# create user model 1
userModel1 = Sequential()
userModel1.add(Dense(30, input_shape=(52,)))
userModel1.add(Activation('tanh'))
userModel1.add(Dropout(0.4))
userModel1.add(Dense(10, input_shape=(30,)))
userModel1.add(Activation('tanh'))
userModel1.add(Reshape((10,1)))

In [ ]:
# create user model 2
userModel2 = Sequential()
userModel2.add(Dense(30, input_shape=(52,)))
userModel2.add(Activation('tanh'))
userModel2.add(Dropout(0.4))
userModel2.add(Dense(10, input_shape=(30,)))
userModel2.add(Activation('tanh'))
userModel2.add(Reshape((10,1)))

In [ ]:
# create article model
articleModel = Sequential()
articleModel.add(Dense(50, input_shape=(181,)))
articleModel.add(Activation('tanh'))
articleModel.add(Dense(30, input_shape=(50,)))
articleModel.add(Activation('tanh'))
articleModel.add(Dense(10, input_shape=(30,)))
articleModel.add(Activation('tanh'))
articleModel.add(Reshape((10,1)))

In [ ]:
model1 = Sequential()
model1.add(Merge([userModel1,articleModel], mode='cos', dot_axes=1, output_shape=(1,)))
model1.add(Reshape((1,)))

In [ ]:
model2 = Sequential()
model2.add(Merge([userModel2,articleModel], mode='cos', dot_axes=1, output_shape=(1,)))
model2.add(Reshape((1,)))

In [ ]:
model=Sequential()
model.add(Merge([model1,model2],mode='concat'))
model.add(Dense(1, input_shape=(2,)))
#model.add(Activation('normalization'))
model.add(Reshape(2))
model.add(Activation('softmax'))

### Calculate Precision and Recall for MV-DSSM

### Try title data

### Apply CNN to Title data

# Build Final Model

# Get Result